In [ ]:
#Import dependencies 
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)

Extract the Data

In [ ]:
#Read the Shark Tank data into a dataframe
sharktank_df = pd.read_csv('shark_tank_raw_data/Shark Tank US dataset.csv')
sharktank_df.head()


Perform initial data cleaning

In [ ]:
sharktank_df.info()

In [ ]:
sharktank_df.columns

In [ ]:
#remove unneccesary columns
sharktank_df_clean = sharktank_df[['Season Number', 'Startup Name', 'Episode Number', 'Pitch Number',
       'Original Air Date', 'Industry',
       'Business Description', 'Company Website', 'Pitchers Gender',
       'Pitchers City', 'Pitchers State', 'Entrepreneur Names', 'Multiple Entrepreneurs',
       'Original Ask Amount', 'Original Offered Equity', 'Valuation Requested',
       'Got Deal', 'Total Deal Amount', 'Total Deal Equity', 'Deal Valuation',
       'Number of Sharks in Deal', 'Investment Amount Per Shark',
       'Equity Per Shark', 'Barbara Corcoran Investment Amount',
       'Barbara Corcoran Investment Equity', 'Mark Cuban Investment Amount',
       'Mark Cuban Investment Equity', 'Lori Greiner Investment Amount',
       'Lori Greiner Investment Equity', 'Robert Herjavec Investment Amount',
       'Robert Herjavec Investment Equity', 'Daymond John Investment Amount',
       'Daymond John Investment Equity', 'Kevin O Leary Investment Amount',
       'Kevin O Leary Investment Equity', 'Guest Investment Amount',
       'Guest Investment Equity', 'Guest Name', 'Barbara Corcoran Present',
       'Mark Cuban Present', 'Lori Greiner Present', 'Robert Herjavec Present',
       'Daymond John Present', 'Kevin O Leary Present', 'Guest Present']]

sharktank_df_clean.head()

In [ ]:
#transform dates to dt format
sharktank_df_clean['Original Air Date'] = pd.to_datetime(sharktank_df_clean['Original Air Date'], format='%d-%b-%y')


In [ ]:
#check that dates converted correctly
sharktank_df_clean.head()

In [ ]:
#transform 1 and 0 values to boolean
sharktank_df_clean[['Got Deal', 
                    'Barbara Corcoran Present', 
                    'Mark Cuban Present', 
                    'Lori Greiner Present', 
                    'Robert Herjavec Present', 
                    'Daymond John Present',
                    'Kevin O Leary Present', 
                    'Guest Present']] = sharktank_df_clean[['Got Deal',
                                                            'Barbara Corcoran Present', 
                                                            'Mark Cuban Present', 
                                                            'Lori Greiner Present', 
                                                            'Robert Herjavec Present', 
                                                            'Daymond John Present', 
                                                            'Kevin O Leary Present', 
                                                            'Guest Present']].astype('bool')

In [ ]:
#check that values converted to boolean correctly
sharktank_df_clean.head()

Pitchers Demographics Table Cleaning (Anna)

Shark Demogrpahics Table Cleaning (Atul)

Pitch Info Table Cleaning (Tianyue)

In [ ]:
sharktank_df_clean.info()

In [ ]:
# Sharks Demographic
Shark_Name = sharktank_df_clean["Guest Name"].dropna().unique()
sharks_demographic_df = pd.DataFrame({'Shark_Name':Shark_Name})
# sharks_demographic_df.index.name="Shark_ID"
sharks_demographic_df.reset_index(inplace=True)
sharks_demographic_df = sharks_demographic_df.rename(columns={'index': 'Shark_ID'})

# gender =  {'Gender':{'M','M','M','M','M','M','M','M','M','M','M','F','F','F','M','M','M','M','F','F','F','M','F','M','F','M','F','F','M','F','M','F','M','M',}}
# Data discrepancy: 26 Nirav Tolia; 27	Nirv Tolia



sharks_demographic_df.loc[27, 'Shark_Name'] = "Nirav Tolia"
# sharks_demographic_df = 
sharks_demographic_df["Shark_Name"].drop_duplicates()
# sharks_demographic_df['Shark_ID'] = range(1, len(sharks_demographic_df) + 1)

sharks_demographic_df

# deal_df.loc[row_index, 'column_name'] = new_value


In [ ]:
# Randomly assign genders to sharks: 'M' for male or 'F' for female
# For reproducibility, we can get the same random assignment each time you run this
# np.random.seed(0)
# sharks_demographic_df['Gender'] = np.random.choice(['Male', 'Female'], size=len(sharks_demographic_df))
gender =  ['Male','Male','Male','Male','Male','Male','Male','Male','Male','Male','Male','Female','Female','Female','Male','Male','Male','Male','Female','Female','Female','Male','Female','Male','Female','Male','Female','Female','Male','Female','Male','Female','Male','Male']
sharks_demographic_df['Gender'] = gender
sharks_demographic_df

In [ ]:
# Creating a dataframe for Deal Info:
deal_df = sharktank_df_clean[[ 'Pitch Number','Got Deal','Total Deal Amount','Total Deal Equity','Original Offered Equity','Valuation Requested', 'Deal Valuation','Investment Amount Per Shark', 'Equity Per Shark','Number of Sharks in Deal', 'Barbara Corcoran Investment Amount',
                    'Barbara Corcoran Investment Equity', 'Mark Cuban Investment Amount','Mark Cuban Investment Equity', 'Lori Greiner Investment Amount','Lori Greiner Investment Equity', 
                    'Robert Herjavec Investment Amount', 'Robert Herjavec Investment Equity','Daymond John Investment Amount','Daymond John Investment Equity', 'Kevin O Leary Investment Amount',
                    'Kevin O Leary Investment Equity', 'Guest Investment Amount', 'Guest Investment Equity']]
 

deal_df.columns = deal_df.columns.str.replace(' ', '_')
deal_df['Deal_ID'] = range(1, len(deal_df) + 1)
deal_df.head(20)


In [ ]:
numeric_columns = deal_df.select_dtypes(include=['float64', 'int64'])
deal_df[numeric_columns.columns] = numeric_columns.fillna(0)
deal_df.head()


In [ ]:
# Use case 1: Filter for successful deals and merge with "Pitch_df" to create visualizations
# Ex. Use Pitch_Number, Total_Deal_Amount, Total_Deal_Equity, Industry
deal_y_df = deal_df[deal_df['Got_Deal']==True]
deal_y_df


pitch_deal_merge_df = pd.merge(deal_y_df, pitch_demo, on="Pitch_Number", how="outer")
pitch_deal_merge_df

In [ ]:
# deal_df.to_csv("deal_df.csv", index=False)
# Only run when exporting

In [ ]:
deal_df.count()

In [ ]:
deal_df.dropna().count()

In [ ]:
deal_df1 = deal_df[~deal_df.Total_Deal_Amount.isnull()].set_index('Pitch_Number')
deal_df1.count()   
deal_df1.head()

In [ ]:
deal_df1.Barbara_Corcoran_Investment_Amount.dropna().sort_values(ascending=False)[:10].plot.bar(figsize=[16,4],rot=90)

In [ ]:
deal_df1.Barbara_Corcoran_Investment_Equity.dropna().sort_values(ascending=False)[:10].plot.bar(figsize=[16,4],rot=90)

In [ ]:
deal_df1.Mark_Cuban_Investment_Amount.dropna().sort_values(ascending=False)[:10].plot.bar(figsize=[16,4],rot=90)


In [ ]:
deal_df1.Mark_Cuban_Investment_Equity.dropna().sort_values(ascending=False)[:10].plot.bar(figsize=[16,4],rot=90)


In [ ]:
#Creating a dataframe for Pitch Info: 
pitch_df= sharktank_df_clean[['Pitch Number', 'Episode Number', 'Season Number','Business Description', 'Got Deal']]
pitch_df.columns = pitch_df.columns.str.replace(' ', '_')
pitch_df.head()

In [ ]:
# pitch_df.to_csv("pitch_df.csv", index=False)
# Only run when exporting